# Coursework Notebook

**Tasks**

* Add Data source
* OOP
* Clean code
* Front-end
* Add filter options 
* Add tests

Imports

In [54]:
# from keplergl import KeplerGl
import shapely
# import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import hvplot
# import httpx
import requests
import json
from shapely.geometry import Polygon, Point
# import rtree
import pathlib
import re
import typing

In [ ]:
string_test = """
123.456.789

affsfdsf

123-446-342

"""

pattern = re.compile(r'\d+\d+\d+')

matches = pattern.finditer(string_test)

print(matches)

In [ ]:
url_test = 'https://data.police.uk/api/crimes-street/burglary?poly=51.705417,-0.559440:51.701934,0.263004:51.301087,0.281818:51.298187,-0.513268&date=2021-07'
pattern = r'(([a-z]+[-][a-z]+)|([a-z]+))\?poly=([-+]?[0-9]*\.?[0-9]*,[-+]?[0-9]*\.?[0-9]*:){2,}[-+]?[0-9]*\.?[0-9]*,[-+]?[0-9]*\.?[0-9]'

match = re.search(pattern, url_test)

matchList = str(match.group()).rstrip(']').split(' ')
print(matchList)
# matches = pattern.finditer(url_test)

# for match in matches:
#   print(match[0])

['burglary?poly=51.705417,-0.559440:51.701934,0.263004:51.301087,0.281818:51.298187,-0.5']


In [ ]:
x = pd.DataFrame()
x.empty

True

In [ ]:
pathlib.Path.home().resolve()

PosixPath('/root')

## TFL Station Locations

Getting stop Ids

In [23]:
# stoppoint = input("Enter a station: ")



{'$type': 'Tfl.Api.Presentation.Entities.SearchResponse, Tfl.Api.Presentation.Entities', 'query': 'CustomHouse', 'total': 2, 'matches': [{'$type': 'Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities', 'icsId': '1001079', 'modes': ['dlr', 'bus'], 'zone': '3', 'id': 'HUBCUS', 'name': 'Custom House', 'lat': 51.518757, 'lon': 0.027418}, {'$type': 'Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities', 'icsId': '1009092', 'topMostParentId': '490G00009092', 'modes': ['bus'], 'id': '490G00009092', 'name': 'Leslie Road / Custom House Station', 'lat': 51.510218, 'lon': 0.026145}]}
HUBCUS


In [39]:
stopID_url = "https://api.tfl.gov.uk/Stoppoint/Search/Royal Victoria DLR Station"

stopID_data = requests.get(stopID_url).json()
print(stopID_data)
available_modes = ['cable-car', 'dlr', 'national-rail', 'overground', 'replacement-bus', 'river-bus', 'river-tour', 'tflrail', 'tram', 'tube']
stopID = stopID_data['matches'][0]['id']
# for dict_ in stopID_data['matches']:
#   if set(dict_['modes']).intersection(set(available_modes)) != set():
#     stopID_array.append(dict_['id'])

# print(stopID_array)
print(stopID)

{'$type': 'Tfl.Api.Presentation.Entities.SearchResponse, Tfl.Api.Presentation.Entities', 'query': 'Royal Victoria DLR Station', 'total': 1, 'matches': [{'$type': 'Tfl.Api.Presentation.Entities.MatchedStop, Tfl.Api.Presentation.Entities', 'parentId': 'HUBRVC', 'icsId': '1002071', 'topMostParentId': 'HUBRVC', 'modes': ['dlr'], 'zone': '3', 'id': '940GZZDLRVC', 'name': 'Royal Victoria DLR Station', 'lat': 51.509336, 'lon': 0.018497}]}
940GZZDLRVC


In [57]:
def seconds_to_minutes(num:int) -> str:
  mins = num//60
  if mins == 0:
    return "Due"
  else:
    return f"{mins} min"

In [78]:
stop_data_url = f'https://api.tfl.gov.uk/StopPoint/{stopID_array[0]}/Arrivals'

stop_data = requests.get(stop_data_url).json()

useless_keys = ['bearing', 'destinationNaptanId', 'id', 'lineId', 'modeName', 'naptanId', 'operationType', 'timing', 'vehicleId']

cleaned_stop_data = [{k: v for k, v in dict_.items() if k not in useless_keys} for dict_ in stop_data]

arrival_df = pd.DataFrame(cleaned_stop_data)
arrival_df['timeToStation'] = arrival_df['timeToStation'].apply(seconds_to_minutes)
arrival_df

,$type,stationName,lineName,platformName,direction,destinationName,timestamp,timeToStation,currentLocation,towards,expectedArrival,timeToLive
0,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",Royal Victoria DLR Station,DLR,Platform 1,outbound,Beckton DLR Station,2022-03-18T11:38:44.7528312Z,5 min,,,2022-03-18T11:44:12Z,2022-03-18T11:44:12Z
1,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",Royal Victoria DLR Station,DLR,Platform 1,outbound,Beckton DLR Station,2022-03-18T11:38:44.7528312Z,15 min,,,2022-03-18T11:54:12Z,2022-03-18T11:54:12Z
2,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",Royal Victoria DLR Station,DLR,Platform 1,outbound,Beckton DLR Station,2022-03-18T11:38:44.7528312Z,25 min,,,2022-03-18T12:04:12Z,2022-03-18T12:04:12Z
3,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",Royal Victoria DLR Station,DLR,Platform 2,inbound,Tower Gateway DLR Station,2022-03-18T11:38:44.7528312Z,10 min,,,2022-03-18T11:49:13Z,2022-03-18T11:49:13Z
4,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",Royal Victoria DLR Station,DLR,Platform 2,inbound,Tower Gateway DLR Station,2022-03-18T11:38:44.7528312Z,19 min,,,2022-03-18T11:58:13Z,2022-03-18T11:58:13Z
5,"Tfl.Api.Presentation.Entities.Prediction, Tfl....",Royal Victoria DLR Station,DLR,Platform 2,inbound,Tower Gateway DLR Station,2022-03-18T11:38:44.7528312Z,29 min,,,2022-03-18T12:08:13Z,2022-03-18T12:08:13Z


In [ ]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

tfl_stations = gpd.read_file('stations.kml', driver='KML')
tfl_stations.geometry = gpd.points_from_xy(tfl_stations.geometry.x, tfl_stations.geometry.y)

DriverError: stations.kml: No such file or directory

In [ ]:
tfl_stations_map = KeplerGl(data={'tfl_stations':tfl_stations})
tfl_stations_map

In [ ]:
tfl_polygon = tfl_stations.unary_union.convex_hull
tfl_polygon

In [ ]:
x, y = tfl_polygon.exterior.coords.xy
print(x,y)

In [ ]:
mode_meta_query = httpx.get("https://api.tfl.gov.uk/Line/Meta/Modes")

In [ ]:
modes = [mode['modeName'] for mode in mode_meta_query.json() if mode['isScheduledService']]
modes.remove('bus')
modes.remove('coach')

In [ ]:
stops = []
for mode in modes:
    mode_status_query = httpx.get(f'https://api.tfl.gov.uk/line/mode/{mode}/status')
    lines = {line['id']: line['name'] for line in mode_status_query.json() if line['modeName']==mode}
    for line_id, line_name in lines.items():
        lines_info = httpx.get(f'https://api.tfl.gov.uk/line/{line_id}/stoppoints')
        result = []
        for line_info in lines_info.json():  
            try:
                result_ = dict(Name=line_info['commonName'], mode=mode, line=line_name, geometry=Point(line_info['lon'], line_info['lat'])) 
            except:
                result_ = lines_info.json()    
            result.append(result_)
        stops.extend(result) 

In [ ]:
stops

In [ ]:
stations = gpd.GeoDataFrame(stops, crs="EPSG:4326", geometry='geometry')
stations['line'] = stations.groupby('Name')['line'].transform(', '.join)
stations = stations.drop_duplicates()

In [ ]:
tfl_stations = KeplerGl(data=stations)
tfl_stations

In [ ]:
x = 0
stopID = []
stoppingpoint = input('Enter the station: ')
url = f'https://api.tfl.gov.uk/Stoppoint/Search/{stoppingpoint}'  
r =  requests.get(url).json()

while x == 0:  
  if r['total'] == 0:
    print('There are no matches for your input')
    stoppingpoint = input('Please re-enter the station: ')
    url = f'https://api.tfl.gov.uk/Stoppoint/Search/{stoppingpoint}' 
    r =  requests.get(url).json()
  else:
    if r['total'] > 1:
      for dicts in r['matches']:
        stopID.append(dicts['id'])
    x = 1

stopID

Enter the station: Custom House


['HUBCUS', '490G00009092']

In [ ]:
url = 'https://api.tfl.gov.uk/Stoppoint/Search/Kennedy'
r =  requests.get(url).json()
stopID = []

for dicts in r['matches']:
  stopID.append(dicts['id'])

stopID

## Crime API

**Info:**


https://data.police.uk/docs/method/crime-street/

lat=52.629729&lng=-1.131592

**Generic API call:**

https://data.police.uk/api/crimes-street/all-crimes?lat=51.5074&lng=0.1278&date=2021-07


all-crimes - can be changed to filter for specific crime types

date=2021-07 - can be changed to filter for specific dates in the form y1y2y3y4/m1m2

lat=51.5074&lng=0.1278 - can be changed to filter for certain neighbourhoods

{y[0]},{x[0]}:{y[1]},{x[1]}:{y[2]},{x[2]}:{y[3]},{x[3]}:{y[4]},{x[4]}:{y[5]},{x[5]}:{y[6]},{x[6]}


51.696161,-0.504971:51.311227,-0.496171:51.691990,0.287200:51.298133,0.221950

Regex testing urls

In [ ]:
example_url = 'https://data.police.uk/api/crimes-street/other-theft?poly=51.705417,-0.559440:51.701934,0.263004:51.301087,0.281818:51.298187,-0.513268&date=2021-07'


Getting crime categories for a chosen date

https://data.police.uk/docs/method/crime-categories/

In [ ]:
categories = 'https://data.police.uk/api/crime-categories?date=2020-12'
r = requests.get(categories).json()
r

cat_names = [r[i]['name'] for i in range(len(r))]
cat_url = [r[i]['url'] for i in range(len(r))]

dates = pd.date_range('2011-08','2021-07', 
              freq='MS').strftime("%Y-%m").tolist()
 
print(cat_url, cat_names, dates)

['all-crime', 'anti-social-behaviour', 'bicycle-theft', 'burglary', 'criminal-damage-arson', 'drugs', 'other-theft', 'possession-of-weapons', 'public-order', 'robbery', 'shoplifting', 'theft-from-the-person', 'vehicle-crime', 'violent-crime', 'other-crime'] ['All crime', 'Anti-social behaviour', 'Bicycle theft', 'Burglary', 'Criminal damage and arson', 'Drugs', 'Other theft', 'Possession of weapons', 'Public order', 'Robbery', 'Shoplifting', 'Theft from the person', 'Vehicle crime', 'Violence and sexual offences', 'Other crime'] ['2011-08', '2011-09', '2011-10', '2011-11', '2011-12', '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06', '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12', '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12', '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12', '2015-01', '2

Allowing a user to choose a date 

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

last_month = datetime.now() - relativedelta(months=1)

today = date.today().strftime("%Y-%m")
print("Today's date:", today)
print(last_month)
# print(f'Data is availbale from 2011-08 to {date.today().strftime("%Y-%m")}')

NameError: ignored

Allowing a user to choose a  crime category for a specific date

In [ ]:
crime_options = {'categories':cat_names}             
crime_options_df = pd.DataFrame.from_dict(crime_options)
print('Here are the following options:')
display(crime_options_df)

cat_choice = input(f'Choose a number from 0 to {len(cat_names)-1} to choose a specific crime category: ')

Here are the following options:


,categories
0,All crime
1,Anti-social behaviour
2,Bicycle theft
3,Burglary
4,Criminal damage and arson
5,Drugs
6,Other theft
7,Possession of weapons
8,Public order
9,Robbery


Choose a number from 0 to 14 to choose a specific crime category: 6


Crime and Date specific API call for London

In [ ]:
url = f'https://data.police.uk/api/crimes-street/{cat_url[int(cat_choice)]}?poly=51.705417,-0.559440:51.701934,0.263004:51.301087,0.281818:51.298187,-0.513268&date=2021-07'
print(url)
r = requests.get(url).json()
print(r)
crime_df = pd.DataFrame(r)
crime_df

https://data.police.uk/api/crimes-street/other-theft?poly=51.705417,-0.559440:51.701934,0.263004:51.301087,0.281818:51.298187,-0.513268&date=2021-07


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,category,location_type,location,context,outcome_status,persistent_id,id,location_subtype,month
0,other-theft,Force,"{'latitude': '51.433069', 'street': {'id': 116...",,{'category': 'Investigation complete; no suspe...,30b5b15bff226afe919bb3c004d002d72a1215acc5cd45...,94186284,,2021-07
1,other-theft,Force,"{'latitude': '51.569605', 'street': {'id': 972...",,{'category': 'Investigation complete; no suspe...,fc013391b42e6c53deaf707c931355e5a3f36fcf78bb37...,94088056,,2021-07
2,other-theft,Force,"{'latitude': '51.516698', 'street': {'id': 956...",,{'category': 'Investigation complete; no suspe...,bf9c28e5db69c28f6601a999aeba03509fd2aa0f6042e3...,93980452,,2021-07
3,other-theft,Force,"{'latitude': '51.448162', 'street': {'id': 851...",,"{'category': 'Unable to prosecute suspect', 'd...",f1e6677d8cd15460f16f8115662053d1812e592b0674cb...,93900909,,2021-07
4,other-theft,Force,"{'latitude': '51.372312', 'street': {'id': 115...",,{'category': 'Investigation complete; no suspe...,ddbeebc1d96366a650cbfc91f219f94f55af9b8a337fd4...,94182416,,2021-07
...,...,...,...,...,...,...,...,...,...
9038,other-theft,Force,"{'latitude': '51.517021', 'street': {'id': 956...",,{'category': 'Investigation complete; no suspe...,3eb8d0fd682c8e4528f10fbe7a2bbf16f47dca6d3cbf8c...,93983215,,2021-07
9039,other-theft,Force,"{'latitude': '51.516286', 'street': {'id': 957...",,{'category': 'Investigation complete; no suspe...,c8ff01442b969ba9479d6017d5b7a2d59ac4a72d709eeb...,93984238,,2021-07
9040,other-theft,Force,"{'latitude': '51.511594', 'street': {'id': 587...",,{'category': 'Investigation complete; no suspe...,e54f89890482a7c023955b4e0559dedf7750e3194cfd68...,94187487,,2021-07
9041,other-theft,Force,"{'latitude': '51.510828', 'street': {'id': 587...",,{'category': 'Investigation complete; no suspe...,bd0ae9d13c157e26a01b585ee31267b97caaf4072e3c7b...,94187220,,2021-07


In [ ]:
len(crime_df)

Creating a geopandas Dataframe by unpacking the dictionary column holding latitudes and longitudes from a pandas Dataframe

In [ ]:
locations = crime_df.location.apply(pd.Series)
crime_gdf = gpd.GeoDataFrame(crime_df, geometry=gpd.points_from_xy(locations.longitude, locations.latitude))
display(locations)
crime_gdf.head()

Plotting map of points using kepler

In [ ]:
crime_map = KeplerGl(height=400)
crime_map.add_data(data=crime_gdf)
crime_map

In [ ]:
crimemap = crime_gdf.hvplot(geo=True,
                            title='Crime in London',
                            marker='x',
                            color='red',
                            size=20,
                            tiles='CartoLight', 
                            hover_cols=["category"])
crimemap

Plotting table showing the number of crimes committed for each type

In [ ]:
counts = crime_df['category'].value_counts()
display(counts)

Plotting histograms on each crimes occurence in a fixed year

In [ ]:
counts.plot(kind='barh')

plt.title('Crimes Committed in each Category')
plt.xlabel('Frequency')
plt.ylabel('Crime type')
plt.show()

# London Boroughs

In [ ]:
lad_url = 'https://opendata.arcgis.com/datasets/5c71e19f21024007b6ce54ef47bb51b1_0.geojson'
lad_shapes = gpd.read_file(lad_url)

In [ ]:
lad_shapes.head()

In [ ]:
lad_name_file = gpd.read_file('https://geoportal1-ons.opendata.arcgis.com/datasets/fe6bcee87d95476abc84e194fe088abb_0'
                              '.geojson')
lad_name_file.head()

In [ ]:
fid_range = [i for i in range(331,364)]
lad_name_ldn = lad_name_file.loc[lad_name_file['FID'].isin(fid_range)]
lad_name_ldn

In [ ]:
lad_shapes_ldn = pd.merge(lad_name_ldn, lad_shapes, on='LAD20CD')
lad_shapes_ldn.head()

In [ ]:
lad_shapes_ldn = lad_shapes_ldn.drop(['LAD20NM_x','LAD20NMW_x','geometry_x','LAD20NMW_y'], axis=1)
lad_shapes_ldn.head()

In [ ]:
lad_shapes_ldn = lad_shapes_ldn.rename(columns={'LAD20NM_y':'Borough','geometry_y':'geometry'})
lad_shapes_ldn.head()

In [ ]:
lad_shapes_ldn.to_csv('SAVE LOCATION')

In [ ]:
with open('/Users/tadios/My_Drive/Programming/Coursework/files/ldn.json') as file:
    ldn_boroughs = pd.DataFrame(json.load(file))
    
ldn_boroughs.head()

In [ ]:
ldn_boroughs = ldn_boroughs.drop(['type'], axis=1)
ldn_boroughs.head()

In [ ]:
ldn_boroughs = ldn_boroughs.features.apply(pd.Series)
ldn_boroughs.head()

In [ ]:
ldn_boroughs_NM = ldn_boroughs.properties.apply(pd.Series)
ldn_boroughs_NM.head()

In [ ]:
ldn_boroughs_geo = ldn_boroughs.geometry.apply(pd.Series)
display(ldn_boroughs_geo.head())
list_of_coords = ldn_boroughs_geo.coordinates.to_list()[0][0]
list_of_coords

In [ ]:
# OUTPUT
#    field_geom_type  field_coords
# 0  Polygon          [[[-96.568927, 46.769008], [-96.5689454, 46.77...

poly_ldn = [Polygon(eval(str(ldn_boroughs_geo.coordinates.iloc[i]))[0]) for i in range(len(ldn_boroughs_geo))]

In [ ]:
ldn_boroughs_NM['geometry'] = gpd.GeoSeries(poly_ldn)
ldn_boroughs = gpd.GeoDataFrame(ldn_boroughs_NM)
display(ldn_boroughs.head())

In [ ]:
ldn_centroids = ldn_boroughs[['Borough', 'geometry']].set_index('Borough').centroid
display(ldn_centroids) 

In [ ]:
borough_list = list(ldn_centroids.index)
borough_list

In [ ]:
mapcheck = KeplerGl(height=400)
mapcheck.add_data(data=ldn_centroids)
mapcheck

In [ ]:
ldn_dict = ldn_boroughs[['Borough', 'geometry']].set_index('Borough').T.to_dict()
print(ldn_dict)
list(ldn_dict.keys()).index('Newham')
ldn_dict['Barking and Dagenham']['geometry']

In [ ]:
bounds = ldn_boroughs.geometry.bounds
bounds

In [ ]:
bounds['Borough'] = ldn_dict.keys()
bounds = bounds.set_index('Borough')
bounds

In [ ]:
a = bounds.astype(str)
a['coordinates_url'] = (
    a['miny'] + ',' + a['minx']
    + ':' +
    a['miny'] + ',' + a['maxx']
    + ':' +
    a['maxy'] + ',' + a['maxx']
    + ':' +
    a['maxy'] + ',' + a['minx'])
a

In [ ]:
for borough in ldn_dict.keys():
    list_of_coords = ldn_boroughs_geo.coordinates.to_list()[list(ldn_dict.keys()).index(borough)][0]
    ldn_dict[borough]['geometry'] = ''.join([f'{list_of_coords[i][0]},{list_of_coords[i][1]}:' for i in range(len(list_of_coords))])[:-1]
ldn_dict['Barking and Dagenham']['geometry']

In [ ]:
type(ldn_boroughs)
ldn_polygon = ldn_boroughs.unary_union.convex_hull

x, y = ldn_polygon.exterior.coords.xy
x,y

In [ ]:
polygon_request = ''.join([f'{y[i]},{x[i]}:' for i in range(len(x))])[:-1]
polygon_request

In [ ]:
points_in_poly = gpd.sjoin(crime_gdf, ldn_boroughs, predicate='within')
points_in_poly.head()

In [ ]:
points_in_poly.loc[points_in_poly['Borough'] == 'Southwark']

In [ ]:
a = points_in_poly[['Borough', 'category']]
a.head()

In [ ]:
b = a.groupby(a.columns.tolist(),as_index=True).size()
b[-1]


In [ ]:
b = b.rename(columns={'size':'Number of occurrences'})
print(b.head())
b.plot(kind='barh')

# Postcode data

In [ ]:
postcode_url = f'https://api.postcodes.io/postcodes/E16 3AA'
postcode_data = requests.get(postcode_url).json()
df = pd.DataFrame(postcode_data)
df

In [ ]:
postcode_dict = {'E16 3AA':Point([df.loc['longitude']['result'], df.loc['latitude']['result']])}
postcode_dict['E16 3AA']
# df.loc['outcode']['result']}

# NHS API

Primary Key: 2b33d8954e5047e4aebd9337ef86985e
Secondary Key: 3d43b93de61545debc8afea377a702c2

In [ ]:
# nhs_data = requests.post('https://api.nhs.uk/service-search/search?api-version=1',
#                         headers={
#         'Content-Type': 'application/json',
#     },)
# print(nhs_data)

In [ ]:
hospital_df = pd.read_csv('/Users/tadios/My_Drive/Programming/Coursework/files/nhs_data/Hospital.csv', encoding="ISO-8859-1", error_bad_lines=False)
hospital_df.head()

In [ ]:
hospital_gdf = gpd.GeoDataFrame(hospital_df, geometry=gpd.points_from_xy(hospital_df.Longitude, hospital_df.Latitude)).drop(columns=['Longitude', 'Latitude'])
hospital_gdf.tail()

In [ ]:
hospitals_in_ldn = gpd.sjoin(hospital_gdf, ldn_boroughs, predicate='within')
hospitals_in_ldn.head()

In [ ]:
pharmacies_df = pd.read_csv('/Users/tadios/My_Drive/Programming/Coursework/files/nhs_data/Pharmacy.csv', encoding="ISO-8859-1", error_bad_lines=False)
pharmacies_df.head()

/opt/miniconda3/envs/Coursework/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 64: expected 23 fields, saw 24\nSkipping line 211: expected 23 fields, saw 24\nSkipping line 251: expected 23 fields, saw 24\nSkipping line 288: expected 23 fields, saw 24\nSkipping line 309: expected 23 fields, saw 24\nSkipping line 540: expected 23 fields, saw 24\nSkipping line 613: expected 23 fields, saw 24\nSkipping line 769: expected 23 fields, saw 24\nSkipping line 912: expected 23 fields, saw 24\nSkipping line 967: expected 23 fields, saw 24\nSkipping line 1231: expected 23 fields, saw 25\nSkipping line 1262: expected 23 fields, saw 24\nSkipping line 1388: expected 23 fields, saw 24\nSkipping line 1424: expected 23 fields, saw 24\nSkipping line 1516: expected 23 fields, saw 24\nSkipping line 1864: expected 23

,OrganisationID,OrganisationCode,OrganisationType,SubType,OrganisationStatus,IsPimsManaged,IsEPSEnabled,OrganisationName,Address1,Address2,...,County,Postcode,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax
13043,FA512,Pharmacy,Community Pharmacy,Visible,True,True,Lords Pharmacy,NaN,Unit 61,The Guineas Shopping Centre,...,Suffolk,CB8 8EQ,52.244797,0.4055977463722229,QJG,Suffolk and North East Essex STP,01638 428022,NaN,https://www.lordspharmacynewmarket.co.uk/,01638 428022
13046,FAP38,Pharmacy,Community Pharmacy,Visible,True,True,LloydsPharmacy Inside Sainsbury's,Church Street,Carlisle,NaN,...,CA2 5TF,54.894725799560547,-2.947054,QHM,Cumbria and North East STP,01228533915,lp5045@lloydspharmacy.co.uk,http://www.lloydspharmacy.com,NaN,NaN
13049,FC826,Pharmacy,Community Pharmacy,Visible,True,True,Rutland Late Night Pharmacy,NaN,45c High Street,NaN,...,LE15 6AJ,52.670024871826172,-0.730286,QK1,Leicester,Leicestershire and Rutland STP,01572 723368,info@rutland-pharmacy.com,http://www.rutlandpharmacy.co.uk,0872 1106511
13054,FD294,Pharmacy,Community Pharmacy,Visible,True,True,Cohens Chemist,33 Hill Street,NaN,NaN,...,LE10 1DS,52.540874481201172,-1.367726,QK1,Leicester,Leicestershire and Rutland STP,01455613100,branch.732@mycohens.net,https://www.cohenschemist.co.uk/,01455610512
13062,FJF07,Pharmacy,Community Pharmacy,Visible,True,True,East Street Pharmacy,East Street Pharmacy,47 East Street,NaN,...,TN9 1LA,51.197410583496094,0.279227,QKS,Kent and Medway STP,01732 770055,NaN,NaN,01732 770500,NaN


In [ ]:
pharmacies_df[pharmacies_df.loc['ParentODSCode']=='London']

KeyError: 'ParentODSCode'

## Weather API

api key: b19414406c70effb25f6b8de609f3945, open weather api

# Housing API

In [ ]:
url = 'https://api.zoopla.co.uk/api/v1/property_listings/'
headers = {
    ''
}
r = requests.get(url=url,
                headers = headers).json()
housing_data = pd.DataFrame(r)
housing_data

## Air Quality API

API key: 5e2d1efec05edb9755bd51a98df8a54298bd4630c3d45241d1112b8c6fdf0c59

In [ ]:
air_quality_gdf = gpd.GeoDataFrame()

# for borough in ldn_centroids:   
url = f'https://api.ambeedata.com/latest/by-lat-lng?lat={ldn_centroids["Newham"].y}&lng={ldn_centroids["Newham"].x}'
headers={
        'Content-Type': 'application/json',
        'x-api-key': '5e2d1efec05edb9755bd51a98df8a54298bd4630c3d45241d1112b8c6fdf0c59'
}
air_quality = requests.get(url,headers=headers).json()
air_quality_df = pd.DataFrame(air_quality)
air_quality_df = air_quality_df.stations.apply(pd.Series)
air_quality_df = air_quality_df.join(air_quality_df.aqiInfo.apply(pd.Series)).drop('aqiInfo', axis=1)
air_quality_gdf = air_quality_gdf.join(air_quality_df)

air_quality_gdf

NameError: name 'ldn_centroids' is not defined

In [ ]:
url = 'http://api.erg.ic.ac.uk/AirQuality/Daily/MonitoringIndex/GroupName=London/Date=2021-07/Json'
r = requests.get(url).json()
print(r)
# air_df = pd.DataFrame(r)
# air_df = air_df.loc['LocalAuthority'].apply(pd.Series)
# air_df

In [ ]:
air = 'https://api.ambeedata.com/history/by-lat-lng?lat=12&lng=73&from=2020-07-13 12:16:44&to=2020-07-18 08:16:44'

air_req = requests.get(url).json()
air_df = pd.DataFrame(air_req)
air_df

In [ ]:
air_quality_df = pd.DataFrame(air_quality)
air_quality_df = air_quality_df.stations.apply(pd.Series)
air_quality_df = air_quality_df.join(air_quality_df.aqiInfo.apply(pd.Series)).drop('aqiInfo', axis=1)
air_quality_df

# New Air Quality API

In [ ]:
latest_london_air_quality = requests.get("https://api.erg.ic.ac.uk/AirQuality/Daily/MonitoringIndex/Latest/GroupName=London/Json")

In [ ]:
local_authority_air_quality = latest_london_air_quality.json()['DailyAirQualityIndex']['LocalAuthority']

In [ ]:
def parse_data_from_sites(result: dict, site: dict, species, local_authority: dict):
    site_name = f"{site['@SiteCode']}{species['@SpeciesCode']}"
    result[site_name] = dict(
        SiteName=site['@SiteName'],
        LocalAuthorityName=local_authority['@LocalAuthorityName'],
        Latitude=site['@Latitude'],
        Longitude=site['@Longitude'],
        Species=str(dict(SpeciesCode=species['@SpeciesCode'],
        SpeciesDescription=species['@SpeciesDescription'],
        AirQualityIndex=species['@AirQualityIndex'],
        AirQualityBand=species['@AirQualityBand']))
    )
    return result

In [ ]:
def parse_data_from_site(result, site, local_authority):
    species = site['Species']
    if isinstance(species, list):
        for single_species in species:
            result = parse_data_from_sites(result, site, single_species, local_authority)
    elif isinstance(species, dict):
        result = parse_data_from_sites(result, site, species, local_authority)
    return result

In [ ]:
result_dict = dict()

for local_authority_data in local_authority_air_quality:
    if 'Site' in local_authority_data:
        sites_data = local_authority_data['Site']
        if isinstance(sites_data, list):
            for site_data in sites_data:
                parse_data_from_site(result_dict, site_data, local_authority_data)
        elif isinstance(sites_data, dict):
                parse_data_from_site(result_dict, sites_data, local_authority_data)

In [ ]:
air_quality_df = pd.DataFrame.from_dict(result_dict).T

In [ ]:
air_quality_df['Species'] = air_quality_df.groupby('SiteName')['Species'].transform(' '.join)

In [ ]:
air_quality_df

,SiteName,LocalAuthorityName,Latitude,Longitude,Species
BG1NO2,Barking and Dagenham - Rush Green,Barking and Dagenham,51.563752,0.177891,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BG2NO2,Barking and Dagenham - Scrattons Farm,Barking and Dagenham,51.529389,0.132857,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BG2PM10,Barking and Dagenham - Scrattons Farm,Barking and Dagenham,51.529389,0.132857,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BQ7NO2,Bexley - Belvedere West,Bexley,51.4946486813055,0.137279111232178,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BQ7O3,Bexley - Belvedere West,Bexley,51.4946486813055,0.137279111232178,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
...,...,...,...,...,...
WMBPM10,Westminster - Oxford Street East,Westminster,51.516066,-0.13516388,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
WMCNO2,Westminster - Cavendish Square,Westminster,51.5168016452062,-0.145657269364411,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
WMCPM10,Westminster - Cavendish Square,Westminster,51.5168016452062,-0.145657269364411,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
WMDNO2,Westminster - Elizabeth Bridge,Westminster,51.4922482333024,-0.147114752900186,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."


In [ ]:
sensor_names = list(air_quality_df['SiteName'])
sensor_names

['Barking and Dagenham - Rush Green',
 'Barking and Dagenham - Scrattons Farm',
 'Barking and Dagenham - Scrattons Farm',
 'Bexley - Belvedere West',
 'Bexley - Belvedere West',
 'Bexley - Belvedere West',
 'Bexley - Belvedere West FDMS',
 'Bexley - Slade Green FIDAS',
 'Bexley - Slade Green FIDAS',
 'Bexley - Belvedere FDMS',
 'Bexley - Slade Green',
 'Bexley - Slade Green',
 'Bexley - Slade Green',
 'Bexley - Belvedere',
 'Bexley - Belvedere',
 'Brent - Ikea',
 'Brent - Ikea',
 'Brent - Ikea',
 'Brent - Neasden Lane',
 'Brent - Neasden Lane',
 'Brent - John Keble Primary School',
 'Brent - John Keble Primary School',
 'Brent - ARK Franklin Primary Academy',
 'Brent - ARK Franklin Primary Academy',
 'Bromley - Harwood Avenue',
 'Bromley - Harwood Avenue',
 'Bromley - Harwood Avenue',
 'Camden - Bloomsbury',
 'Camden - Bloomsbury',
 'Camden - Bloomsbury',
 'Camden - Bloomsbury',
 'Camden - Bloomsbury',
 'Camden - Holborn (Bee Midtown)',
 'City of London - Farringdon Street',
 'City of 

In [ ]:
sensor_names = list(map(lambda x: x.split('-')[-1][1:], sensor_names))
sensor_names

['Rush Green',
 'Scrattons Farm',
 'Scrattons Farm',
 'Belvedere West',
 'Belvedere West',
 'Belvedere West',
 'Belvedere West FDMS',
 'Slade Green FIDAS',
 'Slade Green FIDAS',
 'Belvedere FDMS',
 'Slade Green',
 'Slade Green',
 'Slade Green',
 'Belvedere',
 'Belvedere',
 'Ikea',
 'Ikea',
 'Ikea',
 'Neasden Lane',
 'Neasden Lane',
 'John Keble Primary School',
 'John Keble Primary School',
 'ARK Franklin Primary Academy',
 'ARK Franklin Primary Academy',
 'Harwood Avenue',
 'Harwood Avenue',
 'Harwood Avenue',
 'Bloomsbury',
 'Bloomsbury',
 'Bloomsbury',
 'Bloomsbury',
 'Bloomsbury',
 'Holborn (Bee Midtown)',
 'Farringdon Street',
 'The Aldgate School',
 'The Aldgate School',
 'The Aldgate School',
 'Beech Street',
 'Beech Street',
 'Walbrook Wharf',
 'Norbury',
 'Purley Way A23',
 'Hanger Lane Gyratory',
 'Hanger Lane Gyratory',
 'Horn Lane',
 'Western Avenue',
 'Western Avenue',
 'Acton Vale ',
 'Acton Vale ',
 'Horn Lane TEOM',
 'Bush Hill Park',
 'Derby Road',
 'Bowes Primary Scho

In [ ]:
air_quality_df = air_quality_df.groupby('SiteName').first()

In [ ]:
air_quality_df.loc[f'Barking and Dagenham - {sensor_names[1]}']

LocalAuthorityName                                 Barking and Dagenham
Latitude                                                      51.529389
Longitude                                                      0.132857
Species               {'SpeciesCode': 'NO2', 'SpeciesDescription': '...
Name: Barking and Dagenham - Scrattons Farm, dtype: object

In [ ]:
air_quality_df['Species'].apply(pd.Series)

,0
BG1NO2,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BG2NO2,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BG2PM10,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BQ7NO2,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
BQ7O3,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
...,...
WMBPM10,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
WMCNO2,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
WMCPM10,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."
WMDNO2,"{'SpeciesCode': 'NO2', 'SpeciesDescription': '..."


In [ ]:
air_quality = gpd.GeoDataFrame(air_quality_df, crs="EPSG:4326", 
                              geometry=gpd.points_from_xy(air_quality_df['Longitude'], air_quality_df['Latitude']))

SiteName
Barking and Dagenham - Rush Green           {'SpeciesCode': 'NO2', 'SpeciesDescription': '...
Barking and Dagenham - Scrattons Farm       {'SpeciesCode': 'NO2', 'SpeciesDescription': '...
Bexley - Belvedere                          {'SpeciesCode': 'NO2', 'SpeciesDescription': '...
Bexley - Belvedere FDMS                     {'SpeciesCode': 'PM10', 'SpeciesDescription': ...
Bexley - Belvedere West                     {'SpeciesCode': 'NO2', 'SpeciesDescription': '...
                                                                  ...                        
Westminster - Marylebone Road               {'SpeciesCode': 'NO2', 'SpeciesDescription': '...
Westminster - Oxford Street                 {'SpeciesCode': 'NO2', 'SpeciesDescription': '...
Westminster - Oxford Street East            {'SpeciesCode': 'NO2', 'SpeciesDescription': '...
Westminster - Regent St (Crown Estate)      {'SpeciesCode': 'NO2', 'SpeciesDescription': '...
Westminster - Waterloo Pl (Crown Estate)    {'Speci

In [ ]:
air_quality_map = KeplerGl(height=400)
air_quality_map.add_data(air_quality)
air_quality_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'unnamed': {'index': ['Barking and Dagenham - Rush Green', 'Barking and Dagenham - Scrattons Fa…

In [ ]:
air_quality_url = 'http://api.erg.ic.ac.uk/AirQuality/Hourly/MonitoringIndex/GroupName=London/Json'
air_qual = requests.get(air_quality_url).json()
air_qual_df = pd.DataFrame(air_qual)
air_qual_df

In [ ]:
air_qual_df = air_qual_df.loc['LocalAuthority'].apply(pd.Series)
air_qual_df

In [ ]:
air

# Map

In [ ]:
crime_map = KeplerGl(height=400)
lad_url = 'https://opendata.arcgis.com/datasets/5c71e19f21024007b6ce54ef47bb51b1_0.geojson'
lad_gdf = gpd.read_file(lad_url)
crime_map.add_data(data=lad_gdf, name="state")